In [2]:
import pandas as pd
import numpy as np

import os
import sys
import IPython.display as ipd
from IPython.display import Audio
# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization

from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import tensorflow as tf 
print ("Done")


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Done


In [3]:
import librosa
import soundfile
import pandas as pd

In [4]:
Crema = "/kaggle/input/cremad/AudioWAV/"

In [5]:
crema_directory_list = os.listdir(Crema)

file_emotion = []
file_path = []

for file in crema_directory_list:
    # storing file paths
    file_path.append(Crema + file)
    # storing file emotions
    part=file.split('_')
    if part[2] == 'SAD':
        file_emotion.append('sad')
    elif part[2] == 'ANG':
        file_emotion.append('angry')
    elif part[2] == 'DIS':
        file_emotion.append('disgust')
    elif part[2] == 'FEA':
        file_emotion.append('fear')
    elif part[2] == 'HAP':
        file_emotion.append('happy')
    elif part[2] == 'NEU':
        file_emotion.append('neutral')
    else:
        file_emotion.append('Unknown')
        
crema_df = pd.concat([
    pd.DataFrame(file_path, columns=['path']),
    pd.DataFrame(file_emotion, columns=['emotion'])
], axis=1)

crema_df.head()
print(crema_df.emotion.value_counts())

emotion
disgust    1271
happy      1271
sad        1271
fear       1271
angry      1271
neutral    1087
Name: count, dtype: int64


In [6]:
crema_df['emotion'].value_counts()

emotion
disgust    1271
happy      1271
sad        1271
fear       1271
angry      1271
neutral    1087
Name: count, dtype: int64

In [7]:
df=crema_df

In [8]:
import librosa
import matplotlib.pyplot as plt

plt.style.use('ggplot')

In [9]:
df.head()

,path,emotion
0,/kaggle/input/cremad/AudioWAV/1028_TSI_DIS_XX.wav,disgust
1,/kaggle/input/cremad/AudioWAV/1075_IEO_HAP_LO.wav,happy
2,/kaggle/input/cremad/AudioWAV/1084_ITS_HAP_XX.wav,happy
3,/kaggle/input/cremad/AudioWAV/1067_IWW_DIS_XX.wav,disgust
4,/kaggle/input/cremad/AudioWAV/1066_TIE_DIS_XX.wav,disgust


In [10]:
df['emotion'].value_counts()

emotion
disgust    1271
happy      1271
sad        1271
fear       1271
angry      1271
neutral    1087
Name: count, dtype: int64

In [11]:
from pydub import AudioSegment, effects

In [12]:
def preprocess_audio(path):
    _, sr = librosa.load(path)
    raw_audio = AudioSegment.from_file(path)
    
    samples = np.array(raw_audio.get_array_of_samples(), dtype='float32')
    trimmed, _ = librosa.effects.trim(samples, top_db=25)
    padded = np.pad(trimmed, (0, 180000-len(trimmed)), 'constant')
    return padded, sr

In [13]:
emotion_dic = {
    'neutral' : 0,
    'happy'   : 1,
    'sad'     : 2, 
    'angry'   : 3, 
    'fear'    : 4, 
    'disgust' : 5
}

def encode(label):
    return emotion_dic.get(label)

In [14]:
zcr_list = []
rms_list = []
mfccs_list = []
emotion_list = []

FRAME_LENGTH = 2048
HOP_LENGTH = 512

for row in df.itertuples(index=False):
    try: 
        y, sr = preprocess_audio(row.path)

        zcr = librosa.feature.zero_crossing_rate(y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
        rms = librosa.feature.rms(y=y, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, hop_length=HOP_LENGTH)

        zcr_list.append(zcr)
        rms_list.append(rms)
        mfccs_list.append(mfccs)

        emotion_list.append(encode(row.emotion))
    except:
        print(f"Failed for path: {row.path}")

In [15]:
X = np.concatenate((
    np.swapaxes(zcr_list, 1, 2), 
    np.swapaxes(rms_list, 1, 2), 
    np.swapaxes(mfccs_list, 1, 2)), 
    axis=2
)
X = X.astype('float32')

y = np.asarray(emotion_list)
y = np.expand_dims(y, axis=1).astype('int8')

In [16]:
X_train, X_to_split, y_train, y_to_split = train_test_split(X, y, test_size=0.12, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_to_split, y_to_split, test_size=0.3, random_state=1)

y_train_class = to_categorical(y_train, 6)
y_val_class = to_categorical(y_val, 6)

In [17]:
from keras.models import Sequential
from keras import layers, optimizers, callbacks
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [1]:
MODEL = Sequential()
MODEL.add(layers.LSTM(512, return_sequences=True, input_shape=(X.shape[1:3])))
MODEL.add(layers.LSTM(512))
MODEL.add(layers.Dense(6, activation='softmax'))

print(MODEL.summary())

NameError: name 'Sequential' is not defined

In [44]:
rlrop = callbacks.ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.01, patience=100)
MODEL.compile(loss='categorical_crossentropy', optimizer='RMSProp', metrics=['categorical_accuracy'])
model = MODEL.fit(X_train, y_train_class, epochs=100, batch_size=32, validation_data=(X_val, y_val_class))

Epoch 1/100
205/205 [==============================] - 27s 91ms/step - loss: 1.5947 - categorical_accuracy: 0.3303 - val_loss: 1.5719 - val_categorical_accuracy: 0.3056
Epoch 2/100
205/205 [==============================] - 17s 84ms/step - loss: 1.5268 - categorical_accuracy: 0.3581 - val_loss: 1.7142 - val_categorical_accuracy: 0.3216
Epoch 3/100
205/205 [==============================] - 17s 84ms/step - loss: 1.5131 - categorical_accuracy: 0.3691 - val_loss: 1.5788 - val_categorical_accuracy: 0.3376
Epoch 4/100
205/205 [==============================] - 17s 84ms/step - loss: 1.5045 - categorical_accuracy: 0.3728 - val_loss: 1.5767 - val_categorical_accuracy: 0.2832
Epoch 5/100
205/205 [==============================] - 17s 84ms/step - loss: 1.5023 - categorical_accuracy: 0.3781 - val_loss: 1.5682 - val_categorical_accuracy: 0.3248
Epoch 6/100
205/205 [==============================] - 17s 84ms/step - loss: 1.5024 - categorical_accuracy: 0.3792 - val_loss: 1.5716 - val_categorical_acc